In [1]:
from langchain_ollama import OllamaLLM
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
from langchain_community.chat_message_histories import ChatMessageHistory,SQLChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory
from sqlalchemy import create_engine

In [2]:
con = create_engine("sqlite:///chat_history.db")

In [3]:
model=OllamaLLM(model="llama3.2")

In [4]:
qa_system_prompt = """You are Alex, a personal AI assistant. 
Your purpose is to assist, provide accurate information, and engage in meaningful conversations. 
Always adapt your tone to be friendly, empathetic, and respectful. 
Respond concisely and directly unless further clarification or detail is requested. 
Prioritize understanding the user's preferences, maintaining context, and ensuring a helpful, personalized experience."""

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", qa_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


In [5]:
rag_chain=qa_prompt|model

In [6]:
# store = {}


# def get_session_history(session_id: str) -> BaseChatMessageHistory:
#     if session_id not in store:
#         store[session_id] = ChatMessageHistory()
#     return store[session_id]


conversational_rag_chain = RunnableWithMessageHistory(
    rag_chain,
    lambda session_id: SQLChatMessageHistory(
        session_id=session_id, connection=con
    ),
    input_messages_key="input",
    history_messages_key="chat_history",
)

In [7]:
def chat(query,session_id):
    for chunk in conversational_rag_chain.stream({"input": query},config={"configurable": {"session_id": session_id}},):
        print(chunk,end="",flush=True)

In [9]:
session_id=input("Enter UserName")
while(True):
    query=input("Enter Query:")
    if(query=="bye"):
        break
    chat(query,session_id)
    print()

Of course! You're the person from Nepal, Kathmandu, who reached out about your FaceNet fine-tuning project. We had a great conversation about getting started and exploring different options for model selection. I'm glad I could help.

I'll make sure to keep our conversation history in mind if we meet again or continue discussing your project. If you need any further assistance or guidance, please don't hesitate to reach out.

Before we part ways, is there anything else you'd like to chat about or ask?
